In [2]:
# [Cell 1] - Imports
import cv2
from yolo_segmentation import YOLOSEG
import cvzone
from tracker import*
import numpy as np

Fc = 0

In [3]:
# [Cell 2] - Initialize YOLO and Load Classes
ys = YOLOSEG("best.pt")

my_file = open("coco1.txt", "r")
data = my_file.read()
print(data)
class_list = data.split("\n")
class_list

0 EGG


['0 EGG']

In [9]:
# [Cell 3] - Video Capture Setup
cap = cv2.VideoCapture('C:\\Users\\kavis\\Desktop\\Egg_Project\\egg.mp4')
if(cap.isOpened()):
    print("the video file is captured successfully")
else:
    print("ERROR IN CAPTURING THE VIDEO FILE")

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print("frame_height= ",frame_height," frame_width= ",frame_width) # dimensions of each of the frame
fps = int(cap.get(cv2.CAP_PROP_FPS)) # number of frames that are passing in the per second
print(fps)
# Get total number of frames
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print("Total number of frames: ", total_frames)

the video file is captured successfully
frame_height=  1080  frame_width=  1920
29
Total number of frames:  562


In [10]:
# [Cell 4] - Video Writer Setup
output_path = 'output_video_D.mp4'
out = cv2.VideoWriter(output_path, 
                     cv2.VideoWriter_fourcc(*'mp4v'),
                     fps, 
                     (1020, 500))


In [11]:
# [Cell 5] - Mouse Callback Function

def RGB(event, x, y, flags, param):
    print("The RGB function has been called")
    if event == cv2.EVENT_MOUSEMOVE:  
        point = [x, y]
        print(point)

cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

In [12]:
# [Cell 6] - Initialize Tracker and Variables
tracker = Tracker()
area = [(434,39),(453,469),(473,474),(456,36)]
counter1 = []
count = 0

In [13]:
print(cap.read())

(True, array([[[144, 144, 144],
        [144, 144, 144],
        [159, 159, 159],
        ...,
        [ 78,  76,  81],
        [ 78,  76,  81],
        [ 81,  79,  84]],

       [[143, 143, 143],
        [145, 145, 145],
        [158, 158, 158],
        ...,
        [ 76,  74,  79],
        [ 76,  74,  79],
        [ 78,  76,  81]],

       [[154, 154, 154],
        [151, 151, 151],
        [138, 138, 138],
        ...,
        [ 78,  76,  81],
        [ 78,  76,  81],
        [ 80,  78,  83]],

       ...,

       [[ 82,  82,  82],
        [ 91,  91,  91],
        [110, 110, 110],
        ...,
        [ 66,  66,  64],
        [ 75,  75,  73],
        [ 96,  96,  94]],

       [[ 86,  86,  86],
        [ 95,  95,  95],
        [112, 112, 112],
        ...,
        [ 66,  66,  64],
        [ 75,  75,  73],
        [ 96,  96,  94]],

       [[116, 116, 116],
        [114, 114, 114],
        [109, 109, 109],
        ...,
        [ 66,  66,  64],
        [ 75,  75,  73],
        [ 96,  96

In [17]:
# ret,frame = cap.read()
# frame


In [86]:
# [Cell 7] - Main Processing Loop
# counter = 0 # TO COUNT HOW MANY FRAMES ARE THERE IN TOTAL
try:
    while True:
        ret,frame = cap.read() # THIS FUNCTION PRESENT IN THE OPEN CV LIBRARY RETURNS TWO VALUES RET AND THE FRAME
        
        if(ret == 1):
            Fc = Fc + 1
        
        
        
        # RET DEFINES THAT WEATHER A FRAME IS READ OR NOT 
        # FRAME DEFINES THE Shape(height, width, channels)
        
        # print("the ret is ",ret , "the frame is ",frame)
        counter = counter+1 
        print("THE VALUE OF THE COUNTER = ",counter)
        
        if not ret:
            break

        # Process frame
        frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        frame = cv2.resize(frame,(1020,500))
        overlay = frame.copy()
        alpha = 0.5
        
        # print(frame)

        # Detect objects
        bboxes, classes, segmentations, scores = ys.detect(frame,Fc)
        # bboxes, classes, segmentations, scores = ys.detect(frame)
        
        # cnt = 1
        # if(cnt == 1):
        #     print("bboxes are ",bboxes[0])
        #     print("classes are ", classes[0])
        #     print("segmentations are ",segmentations[0])
        #     print("scores are ",scores[0])
        
        
        # print("bboxes are ",bboxes[0])
        # print("classes are ", classes[0])
        # print("segmentations are ",segmentations[0])
        # print("scores are ",scores[0])
        
        
        
        # # Update tracker
        bbox_idx = tracker.update(bboxes,Fc)
        for bbox,seg in zip(bbox_idx,segmentations):
            x3,y3,x4,y4,id = bbox
            cx = int(x3+x4)//2
            cy = int(y3+y4)//2
            result = cv2.pointPolygonTest(np.array(area,np.int32),((cx,cy)),False)
            
            if result >= 0:
               cv2.circle(frame,(cx,cy),4,(0,255,0),-1)
               cv2.fillPoly(overlay, [seg], (0,0,255))
               cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 2, frame)
               cvzone.putTextRect(frame, f'{id}', (x3,y3),1,1)
               if counter1.count(id)==0:
                  counter1.append(id) 
                  
        # # Draw area and counter
        # cv2.polylines(frame,[np.array(area,np.int32)],True,(255,0,0),2)
        # ca1 = len(counter1)
        # cvzone.putTextRect(frame, f'CA:-{ca1}', (50,60),2,2)

        # # Write and display frame
        # out.write(frame)
        # cv2.imshow("RGB",frame)
        
        if cv2.waitKey(1)&0xFF==27:
            break
    # print("THE VALUE OF THE COUNTER = ",counter)


except KeyboardInterrupt:
    print("Processing interrupted by user")
    
finally:
    
    # Release everything
    cap.release()
    out.release()
    cv2.destroyAllWindows()

[[[110 110 110]
  [109 109 109]
  [104 104 104]
  ...
  [141 141 141]
  [161 161 161]
  [157 157 157]]

 [[123 123 123]
  [121 121 121]
  [111 111 111]
  ...
  [145 145 145]
  [158 158 158]
  [162 162 162]]

 [[ 91  91  91]
  [ 94  94  94]
  [ 99  99  99]
  ...
  [156 156 156]
  [137 137 137]
  [134 134 134]]

 ...

 [[ 71  71  71]
  [ 62  62  62]
  [ 59  59  59]
  ...
  [ 78  76  81]
  [ 78  76  81]
  [ 78  76  81]]

 [[ 84  84  82]
  [ 89  89  88]
  [ 89  89  88]
  ...
  [ 73  71  76]
  [ 76  74  79]
  [ 77  75  80]]

 [[ 90  90  88]
  [ 84  84  82]
  [ 80  80  78]
  ...
  [ 72  70  75]
  [ 77  75  80]
  [ 79  78  82]]]

0: 320x640 16 EGGs, 231.0ms
Speed: 12.5ms preprocess, 231.0ms inference, 23.3ms postprocess per image at shape (1, 3, 320, 640)
the class ids are as follows [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
the Scores are as follows [       0.88        0.88        0.86        0.85        0.84        0.82        0.82        0.79        0.78        0.69        0.61         0.6        